In [1]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
gcc (GCC) 12.1.0
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/home/abs/anaconda3/envs/open-mmlab3/bin/python


In [2]:
!pwd
%cd ..
!pwd

/run/media/abs/New Volume/B3/mmpose/demo
/run/media/abs/New Volume/B3/mmpose
/run/media/abs/New Volume/B3/mmpose


In [3]:
# Check Pytorch installation
import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)


# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 1.8.2 True
torchvision version: 0.9.2
mmpose version: 0.23.0
cuda version: 11.1
compiler information: GCC 7.3


In [4]:
import numpy as np
import cv2
import joblib
import csv
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
local_runtime = False

try:
  from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
  local_runtime = True

seg_config_file = '/home/abs/mmsegmentation/configs/deeplabv3plus/deeplabv3plus_r101-d8_512x512_40k_voc12aug.py'
seg_checkpoint_file = '/home/abs/Documents/B3 All/Models/Seg/iter_40000.pth'

rear_pose_config = '/home/abs/mmpose/configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/animalpose/res152_animalpose_256x256.py'
rear_pose_checkpoint = '/home/abs/mmpose/work_dirs_rear/res152_animalpose_256x256/epoch_210.pth'
side_pose_config = '/home/abs/mmposeSide_b3/configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/animalpose/res152_animalpose_256x256.py'
side_pose_checkpoint = '/home/abs/mmposeSide_b3/work_dirs/res152_animalpose_256x256/epoch_210.pth'
det_config = '/run/media/abs/New Volume/B3/mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
weight_filename = "/home/abs/mmsegmentation/work_dirs/weight_model/alpha_model_april_2022.joblib"
# initialize seg & pose model
# build the model from a config file and a checkpoint file
model = init_segmentor(seg_config_file, seg_checkpoint_file, device='cuda:0')

rear_pose_model = init_pose_model(rear_pose_config, rear_pose_checkpoint)
side_pose_model = init_pose_model(side_pose_config, side_pose_checkpoint)

# initialize detector
rear_det_model = init_detector(det_config, det_checkpoint)
side_det_model = init_detector(det_config, det_checkpoint)

rear_filename = '/run/media/abs/New Volume/B3/mmpose/tests/data/cow/Rear/rear_imgs_2601.csv'

with open(rear_filename, 'r') as csvrear:
    datareader = csv.reader(csvrear)
    for rear_fname in datareader:
#         print(type(rear_fname))
        rear_fname = ''.join(rear_fname)
        print(rear_fname)
        rear_img = f'/run/media/abs/New Volume/B3/mmpose/tests/data/cow/Rear/images/{rear_fname}'
        rear_mmdet_results = inference_detector(rear_det_model, rear_img)
        rear_person_results = process_mmdet_results(rear_mmdet_results, cat_id=20)
        side_fname = rear_fname.replace('r','s')
        side_img = f'/run/media/abs/New Volume/B3/mmpose/tests/data/cow/Side/images/{side_fname}'
        side_mmdet_results = inference_detector(side_det_model, side_img)
        side_person_results = process_mmdet_results(side_mmdet_results, cat_id=20)

        if side_person_results is not None:
        # inference Segmentation
            

            # inference pose
            rear_pose_results, rear_returned_outputs = inference_top_down_pose_model(rear_pose_model,
                                                                           rear_img,
                                                                           rear_person_results,
                                                                           bbox_thr=0.3,
                                                                           format='xyxy',
                                                                           dataset=rear_pose_model.cfg.data.test.type)
            side_pose_results, side_returned_outputs = inference_top_down_pose_model(side_pose_model,
                                                                           side_img,
                                                                           side_person_results,
                                                                           bbox_thr=0.3,
                                                                           format='xyxy',
                                                                           dataset=side_pose_model.cfg.data.test.type)

            # KPT rear and side
            try:
                rear_kpt = rear_pose_results[0]["keypoints"][:,0:2]
                side_kpt = side_pose_results[0]["keypoints"][:,0:2]
            #     print(side_kpt)

                rearKptID=rearx0=reary0=rearx1=reary1=rearx2=reary2=rearx3=reary3=0
                sideKptID=sidex0=sidey0=sidex1=sidey1=sidex2=sidey2=sidex3=sidey3=sidex4=sidey4=sidex5=sidey5=sidex6=sidey6=sidex7=sidey7=sidex8=sidey8=0

                for kptx,kpty in rear_kpt:
                  if rearKptID == 0:
                      rearx0 = kptx
                      reary0 = kpty
                  elif rearKptID == 1:
                      rearx1 = kptx
                      reary1 = kpty
                  elif rearKptID == 2:
                      rearx2 = kptx
                      reary2 = kpty
                  elif rearKptID == 3:
                      rearx3 = kptx
                      reary3 = kpty

                  rearKptID+=1

                for kptx,kpty in side_kpt:

                  if sideKptID == 0:
                      sidex0 = kptx
                      sidey0 = kpty
                  elif sideKptID == 1:
                      sidex1 = kptx
                      sidey1 = kpty
                  elif sideKptID == 2:
                      sidex2 = kptx
                      sidey2 = kpty
                  elif sideKptID == 3:
                      sidex3 = kptx
                      sidey3 = kpty
                  elif sideKptID == 4:
                      sidex4 = kptx
                      sidey4 = kpty
                  elif sideKptID == 5:
                      sidex5 = kptx
                      sidey5 = kpty 
                  elif sideKptID == 6:
                      sidex6 = kptx
                      sidey6 = kpty
                  elif sideKptID == 7:
                      sidex7 = kptx
                      sidey7 = kpty
                  elif sideKptID == 8:
                      sidex8 = kptx
                      sidey8 = kpty 

                  sideKptID+=1

                side_Length_wither = round(((sidey1-sidey0)**2+(sidex1-sidex0)**2)**0.5)
                side_Length_pinbone = round(((sidey2-sidey1)**2+(sidex2-sidex1)**2)**0.5)
                side_F_Girth = round(((sidey4-sidey3)**2+(sidex4-sidex3)**2)**0.5)
                side_R_Girth = round(((sidey8-sidey7)**2+(sidex8-sidex7)**2)**0.5)
                side_height = round(((sidey6-sidey5)**2+(sidex6-sidex5)**2)**0.5)
                rear_width = round(((reary1-reary0)**2+(rearx1-rearx0)**2)**0.5)
                rear_height = round(((reary3-reary2)**2+(rearx3-rearx2)**2)**0.5)
                print(rear_width)
                scaling_factor = side_height/rear_height
                actual_width = rear_width*scaling_factor
#                 print("sahfdsjfh")
                print(f'aw = {actual_width}')
                print(side_height)
                
                side_seg_result = inference_segmentor(model, side_img)
#                 print(f'side_seg_result= {side_seg_result}')
                rear_seg_result = inference_segmentor(model, rear_img)

                side_seg = np.asarray(side_seg_result)
                rear_seg = np.asarray(rear_seg_result) 
                sticker = cattle = bg = 0


                sticker = (side_seg == 2).sum()

                cattle = (side_seg == 1).sum()

                bg =( side_seg == 0).sum()
                
                cattle_rear = (rear_seg == 1).sum()
                
                ratio = cattle/sticker
                


#                 print(sticker)
#                 print(cattle)
#                 print(cattle_rear)
#                 print(bg)
                with open('/run/media/abs/New Volume/B3/mmpose/hard_features_v3.csv', 'a') as f:
                    writer = csv.writer(f)

                    data = [rear_fname, side_Length_wither, side_Length_pinbone, side_F_Girth, side_R_Girth, side_height, rear_width, rear_height, actual_width,ratio, sticker, cattle, bg] 

    #                 data.append(rear_fname, side_Length_wither, side_Length_pinbone, side_F_Girth, side_R_Girth, side_height, rear_width, rear_height, actual_width, sticker, cattle, bg )
                    writer.writerow(data)
                print(f'ratio = {cattle/sticker} sticker = {sticker} cattle = {cattle} side_Height = {side_height} side_Length_wither = {side_Length_wither}, side_Length_pinbone = {side_Length_pinbone}, side_F_Girth = {side_F_Girth}, side_R_Girth = {side_R_Girth}, rear_Width = {rear_width}, rear_Height = {rear_height}')


                # Weight prediction
                
                loaded_model = joblib.load(weight_filename)
                print(loaded_model.predict([[side_Length_pinbone,	side_F_Girth,	side_R_Girth,	actual_width,	sticker,	cattle]]))
                gc.collect()


                rear_vis_result = vis_pose_result(rear_pose_model,
                             rear_img,
                             rear_pose_results,
                             thickness=1,
                             dataset=rear_pose_model.cfg.data.test.type,
                             show=False)
                side_vis_result = vis_pose_result(side_pose_model,
                                             side_img,
                                             side_pose_results,
                                             thickness=1,
                                             dataset=side_pose_model.cfg.data.test.type,
                                             show=False)
                # reduce image size
                rear_vis_result = cv2.resize(rear_vis_result, dsize=None, fx=1, fy=1)
                side_vis_result = cv2.resize(side_vis_result, dsize=None, fx=1, fy=1)


                if local_runtime:
                  from IPython.display import Image, display
                  import tempfile
                  import os.path as osp
                  with tempfile.TemporaryDirectory() as tmpdir:
                    rear_file_name = osp.join(tmpdir, 'rear_pose_results.png')
                    side_file_name = osp.join(tmpdir, 'side_pose_results.png')

                    cv2.imwrite(rear_file_name, rear_vis_result)
                    cv2.imwrite(side_file_name, side_vis_result)

                    display(Image(rear_file_name))
                    display(Image(side_file_name))
                    show_result_pyplot(model, side_img, side_seg_result, get_palette('voc'))
                    show_result_pyplot(model, rear_img, rear_seg_result, get_palette('voc'))



                else:
                  show_result_pyplot(model, side_img, seg_result, get_palette('voc'))
                  cv2_imshow(rear_vis_result)
                  cv2_imshow(side_vis_result)

    
            except:
                pass

                # show pose estimation results



/home/abs/Documents/B3 All/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /home/abs/Documents/B3 All/Models/Seg/iter_40000.pth
load checkpoint from local path: /home/abs/mmpose/work_dirs_rear/res152_animalpose_256x256/epoch_210.pth
load checkpoint from local path: /home/abs/mmposeSide_b3/work_dirs/res152_animalpose_256x256/epoch_210.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
100_r_109_M.jpg
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
339
aw = 283.8501838235294
911
ratio = 69.58774038461539 sticker = 9984 cattle = 694764 side_Height = 911 side_Length_wither = 932, side_Length_pinbone = 975, side_F_Girth = 462, side_R_Girth = 448, rear_Width = 339, rear_Height = 1088
[139.46]
100_r_131_F.jpg
<class 'numpy.ndarray'>


/home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


<class 'numpy.ndarray'>
363
aw = 349.932
964
ratio = 68.00238641578706 sticker = 10895 cattle = 740886 side_Height = 964 side_Length_wither = 945, side_Length_pinbone = 1021, side_F_Girth = 474, side_R_Girth = 460, rear_Width = 363, rear_Height = 1000
[167.49]
100_r_181_F.jpg
<class 'numpy.ndarray'>


/home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


...

/home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


<class 'numpy.ndarray'>
328
aw = 332.1640625
1037
ratio = 119.26751592356688 sticker = 7850 cattle = 936250 side_Height = 1037 side_Length_wither = 1127, side_Length_pinbone = 1221, side_F_Girth = 536, side_R_Girth = 519, rear_Width = 328, rear_Height = 1024
[238.01]
30_r_110_F.jpg


/home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
355
aw = 307.7909011373578
991
ratio = 81.44745730000965 sticker = 10363 cattle = 844040 side_Height = 991 side_Length_wither = 1020, side_Length_pinbone = 1114, side_F_Girth = 488, side_R_Girth = 455, rear_Width = 355, rear_Height = 1143
[178.25]


/home/abs/anaconda3/envs/open-mmlab3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
